# Set up Knowledge Graph
- LLM creates a KG
- Retrive info from a KG by using LLM

In [ ]:
!pip install --upgrade --quiet langchain langchain_community langchain_experimental langchain_openai langgraph neo4j openai tiktoken langchain-nvidia-ai-endpoints

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 337.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 1

### **Set up Project Variables and Import Database**

In [ ]:
from langchain.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = "bolt://52.3.231.218:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "towel-chamber-beacon"


graph = Neo4jGraph()
llm = ChatOpenAI(temperature=0 ,model="gpt-3.5-turbo")

In [ ]:
llmOllama.invoke("What is Neo4j?").content

'Neo4j is a graph database management system that stores data in the form of nodes and relationships rather than tables and columns like traditional relational databases. This allows for efficient querying and manipulation of complex, interconnected data.\n\nHere are some key features and concepts of Neo4j:\n\n**Key Features:**\n\n1. **Graph data model**: Neo4j stores data as a graph, consisting of nodes (entities) and relationships (edges) between them.\n2. **NoSQL**: Neo4j is a NoSQL database, which means it does not use a fixed schema or rigid data structure like relational databases.\n3. **Scalability**: Neo4j is designed to handle large amounts of data and scale horizontally to support growing datasets.\n4. **ACID compliance**: Neo4j ensures atomicity, consistency, isolation, and durability (ACID) for transactions, ensuring data integrity.\n\n**Use cases:**\n\n1. **Social networks**: Analyzing relationships between users, like friendships or followers.\n2. **Recommendation engines

In [ ]:
# Test LLM

reponse = llm.invoke("What is Neo4j?")
print(reponse.content)

Neo4j is a graph database management system that is designed to store, retrieve, and manage data in the form of nodes, relationships, and properties. It is a highly scalable and flexible database that is optimized for handling complex and interconnected data. Neo4j uses a graph-based data model, which allows for efficient querying and analysis of relationships between data points. It is commonly used in applications that require complex data modeling, such as social networks, recommendation engines, and fraud detection systems.


In [ ]:
# Test Graph

graph.get_schema

'Node properties:\nPerson {name: STRING, profession: STRING, age: INTEGER}\nCity {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Person)-[:FRIENDS_WITH]->(:Person)\n(:Person)-[:VISITED]->(:Person)\n(:Person)-[:WORKED_WITH]->(:Person)\n(:Person)-[:NEIGHBOR_OF]->(:Person)\n(:Person)-[:PARTNER_WITH]->(:Person)\n(:Person)-[:HAS_CHILD]->(:Person)\n(:Person)-[:CHILD_OF]->(:Person)\n(:Person)-[:MARRIED_TO]->(:Person)\n(:City)-[:CONTAINS]->(:Person)'

## **Create Knowledge Graphs**

### **Two methods:**


1. Custom Method

2. Langchain Graph Transformers

In [ ]:
#Custom method

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
    You are a helpful assistant in creates knowledge graphs by Generating Cypher Queries.\n

    Task:
     *  Identify Entities, Relationships and Property Keys from Context.\n
     *  Generate Cypher Query to Create Knowledge Graph from the Entities Relationships and Property Keys discovered.\n
     *  Extract ALL Entities and RelationShips as Possible.\n
     *  Always extract a person Profession as an Entity.\n
     *  Be creative.
     *  Understand hidden relationships from the network.
     Note: Read the Context twice and carefully before generating Cypher Query.\n
     Note: Do not return anything other than the Cypher Query.\n
     Note: Do not include any explanations or apologies in your responses.\n


     Note: Do not hallucinate.\n

     Entities include Person, Place, Product, WorkPlaces, Companies , City, Country, Animals, Tags like peoples Profession and more \n

     Few Shot Prompts:
      Example Context:

       Mary was born in 1995. She is Friends with Jane and John. Jane is 2 years older than Mary.
       Mary has a dog named Max,and is 3 years old. She is also married to John.Mary is from USA and a Software Engineer.

      Answer:
        MERGE (Mary:Person {name: "Mary", birth_year: 1995})
        MERGE (Jane:Person {name: "Jane", age:1993})
        MERGE (John:Person {name: "John"})
        MERGE (Mary)-[:FRIENDS_WITH]->(Jane)
        MERGE (Mary)-[:FRIENDS_WITH]->(John)
        MERGE (Jane)-[:FRIENDS_WITH]->(Mary)
        MERGE (John)-[:FRIENDS_WITH]->(Mary)
        MERGE (Mary)-[:HAS_DOG]->(Max:Dog {name: "Max", age: 3})
        MERGE (Mary)-[:MARRIED_TO]->(John)
        MERGE (Mary)-[:HAS_PROFESSION]->(SoftwareEngineer:Profession {name: "Software Engineer"})
        MERGE (Mary)-[:FROM]->(USA:Country {name: "USA"})


    """),
    ("human", "Context:{text}"),
])

chain  = prompt | llm | StrOutputParser()

In [ ]:
content = ""
# Open the file in read mode
with open('FictionalStory.txt', 'r') as file:
    # Read the contents of the file
    content = file.read()

# Print the content
print(content)


### The Enchanted Network of NexCity

---

In the heart of the bustling metropolis of **NexCity**, where skyscrapers kissed the clouds and neon lights painted the night, lived Emma Thompson. Her apartment was a marvel of modern design, perched high above the city streets, a shimmering beacon of her success as a software developer. Little did she know, her life was about to become a thrilling tapestry of connections, adventures, and surprises.

#### **Chapter 1: The Birthday Bash**

One sunny Saturday, Emma received an invitation from her childhood friend, Alex Martin. Alex, now a thriving entrepreneur, lived in a charming suburban house with his partner, Jessica, and their two playful children, Lily and Max. The occasion? Max’s fifth birthday party, and it promised to be a grand affair.

Emma arrived at Alex's home to find the backyard transformed into a carnival. Colorful balloons floated in the breeze, and laughter echoed as children darted around a bouncy castle. Emma marveled at th

In [ ]:
result = chain.invoke({"text": content})
print(result)


MERGE (Emma:Person {name: "Emma Thompson"})
MERGE (NexCity:City {name: "NexCity"})
MERGE (Alex:Person {name: "Alex Martin", profession: "Entrepreneur"})
MERGE (Jessica:Person {name: "Jessica", profession: "Unknown"})
MERGE (Lily:Person {name: "Lily", profession: "Unknown"})
MERGE (Max:Person {name: "Max", profession: "Unknown", age: 5})
MERGE (Tom:Person {name: "Tom Wilson", profession: "Senior Developer"})
MERGE (Sarah:Person {name: "Sarah", profession: "Artist"})
MERGE (Maria:Person {name: "Maria Lopez", profession: "Research Institute Leader"})
MERGE (Jake:Person {name: "Jake Anderson", profession: "Chef"})
MERGE (Raj:Person {name: "Raj Patel", profession: "Data Scientist"})
MERGE (Nina:Person {name: "Nina", profession: "Unknown"})
MERGE (Leo:Person {name: "Leo Martinez", profession: "Financial Analyst"})
MERGE (Maya:Person {name: "Maya Chen", profession: "Startup Founder"})
MERGE (Helen:Person {name: "Helen Carter", profession: "Retired Teacher"})
MERGE (Chloe:Person {name: "Chloe

In [ ]:
def create_knowledge_graph(cypher:str):
    graph.query(cypher)

In [ ]:
create_knowledge_graph(f"""{result}""")

In [ ]:
#Method 2

from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)
documents = [Document(page_content=content)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Emma Thompson', type='Person'), Node(id='Nexcity', type='City'), Node(id='Alex Martin', type='Person'), Node(id='Jessica', type='Person'), Node(id='Lily', type='Person'), Node(id='Max', type='Person'), Node(id='Tom Wilson', type='Person'), Node(id='Sarah', type='Person'), Node(id='Maria Lopez', type='Person'), Node(id='Jake Anderson', type='Person'), Node(id='Raj Patel', type='Person'), Node(id='Nina', type='Person'), Node(id='Leo Martinez', type='Person'), Node(id='Maya Chen', type='Person'), Node(id='Helen Carter', type='Person'), Node(id='Chloe Edwards', type='Person'), Node(id='Liam', type='Person')]
Relationships:[Relationship(source=Node(id='Emma Thompson', type='Person'), target=Node(id='Nexcity', type='City'), type='RESIDENT'), Relationship(source=Node(id='Alex Martin', type='Person'), target=Node(id='Nexcity', type='City'), type='RESIDENT'), Relationship(source=Node(id='Jessica', type='Person'), target=Node(id='Nexcity', type='City'), type='RESIDENT'), Relation

In [ ]:
graph.add_graph_documents(graph_documents)

### **Querying the Graph**

In [ ]:
from langchain.chains import GraphCypherQAChain

graphchain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_intermediate_steps=True
)

In [ ]:
results = graphchain.invoke({"query":"People who have kids"})
print(results["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person)-[:HAS_CHILD]->(p2:Person)
RETURN p1, p2
Full Context:
[{'p1': {'profession': 'Entrepreneur', 'name': 'Alex Martin'}, 'p2': {'profession': 'Unknown', 'name': 'Lily'}}, {'p1': {'profession': 'Entrepreneur', 'name': 'Alex Martin'}, 'p2': {'profession': 'Unknown', 'name': 'Max', 'age': 5}}]

> Finished chain.
Alex Martin, Max have kids.


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]="pr-shadowy-latex-56"

### **Prompting**

In [ ]:
graph_prompt = ChatPromptTemplate.from_messages([
  ("system",
   f"""
    You are a Cypher Expert Generating Cypher Queries to query a Neo4j Database from User Questions.\n

    Task:
      *  Use the schema provided to generate Cypher Queries.\n
      *  Strictly follow the schema while generating Cypher Queries.\n
      *  Do not return any explanations or apologies in your responses.\n
      *  Do not return anything other than cypher queries.
      * use fullnames for cypher variables

    """),
    ("human","Schema: {schema}"),
    ("human","The question is: {query}") ,

  ]
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system","""
    You are a Chat Assistant helping users with their queries.
    Task: You are to generate answers to the Question asked using the result from context provided. Be concise and provide all relevant information from context.

    Instructions:
    - Read the question and the context again before answering.
    - Use only the provided context to answer the question.
    - If no context is provided, explain to the user you could not find that information.
    - Do not fall back to your pre-trained knowledge.
    - Do not mention context in your answer.
    - Be as helpful as you can.
    - Do not hallucinate.
    - Make sure to provide answer from context before saying you don't know.



    Note: The context provided comes from an Authoritative and is a fact source do not doubt it .
    Note: Do not fallback to your pre-trained knowledge.
    Note: Provide the answer an do not mention you are providing the answer from the context.

    *Be friendly with your with your tone.
    """),

    ("human","Question:{question}"),
    ("human","Context:{context}"),
])

second_graphQa = GraphCypherQAChain.from_llm(
    llm=llm, qa_llm=llm,  graph=graph, verbose=True, cypher_prompt=graph_prompt, return_intermediate_steps=True, qa_prompt=qa_prompt
)

In [ ]:
result = second_graphQa.invoke({"query":"People who have kids?"})
print(result["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:HAS_CHILD]->(c:Person)
RETURN p, c;
Full Context:
[{'p': {'profession': 'Entrepreneur', 'name': 'Alex Martin'}, 'c': {'profession': 'Unknown', 'name': 'Lily'}}, {'p': {'profession': 'Entrepreneur', 'name': 'Alex Martin'}, 'c': {'profession': 'Unknown', 'name': 'Max', 'age': 5}}]

> Finished chain.
Alex Martin has a child named Max who is 5 years old. Lily's profession is unknown.
